# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 22 2022 8:30AM,242484,10,MSP212432,"Methapharm, Inc.",Released
1,Jul 22 2022 8:30AM,242484,10,MSP212425,"Methapharm, Inc.",Released
2,Jul 22 2022 8:30AM,242486,10,PRF-33782,Bio-PRF,Released
3,Jul 22 2022 8:30AM,242486,10,PRF-33785,Bio-PRF,Released
4,Jul 22 2022 8:30AM,242486,10,PRF-33791,Bio-PRF,Released
5,Jul 22 2022 8:30AM,242486,10,PRF-33794,Bio-PRF,Released
6,Jul 21 2022 3:33PM,242487,12,HH6452,Hush Hush,Executing
7,Jul 21 2022 3:13PM,242481,10,0085899564,ISDIN Corporation,Released
8,Jul 21 2022 3:13PM,242481,10,0085899254,ISDIN Corporation,Released
9,Jul 21 2022 3:12PM,242479,10,0085899193,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,242480,Executing,3
33,242481,Released,2
34,242484,Released,2
35,242486,Released,4
36,242487,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242480,NaN,3.0,NaN
242481,NaN,NaN,2.0
242484,NaN,NaN,2.0
242486,NaN,NaN,4.0
242487,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242348,20.0,33.0,24.0
242375,0.0,0.0,1.0
242399,0.0,0.0,1.0
242408,0.0,0.0,1.0
242410,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242348,20,33,24
242375,0,0,1
242399,0,0,1
242408,0,0,1
242410,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242348,20,33,24
1,242375,0,0,1
2,242399,0,0,1
3,242408,0,0,1
4,242410,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242348,20,33,24
1,242375,,,1
2,242399,,,1
3,242408,,,1
4,242410,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc."
2,Jul 22 2022 8:30AM,242486,10,Bio-PRF
6,Jul 21 2022 3:33PM,242487,12,Hush Hush
7,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation
9,Jul 21 2022 3:12PM,242479,10,ISDIN Corporation
33,Jul 21 2022 3:09PM,242480,19,"AdvaGen Pharma, Ltd"
36,Jul 21 2022 3:03PM,242476,10,ISDIN Corporation
59,Jul 21 2022 2:53PM,242477,10,Emerginnova
60,Jul 21 2022 2:51PM,242473,10,ISDIN Corporation
101,Jul 21 2022 2:49PM,242475,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc.",,,2
1,Jul 22 2022 8:30AM,242486,10,Bio-PRF,,,4
2,Jul 21 2022 3:33PM,242487,12,Hush Hush,,1,
3,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation,,,2
4,Jul 21 2022 3:12PM,242479,10,ISDIN Corporation,,,24
5,Jul 21 2022 3:09PM,242480,19,"AdvaGen Pharma, Ltd",,3,
6,Jul 21 2022 3:03PM,242476,10,ISDIN Corporation,,,23
7,Jul 21 2022 2:53PM,242477,10,Emerginnova,,,1
8,Jul 21 2022 2:51PM,242473,10,ISDIN Corporation,,,41
9,Jul 21 2022 2:49PM,242475,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc.",2,,
1,Jul 22 2022 8:30AM,242486,10,Bio-PRF,4,,
2,Jul 21 2022 3:33PM,242487,12,Hush Hush,,1,
3,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation,2,,
4,Jul 21 2022 3:12PM,242479,10,ISDIN Corporation,24,,
5,Jul 21 2022 3:09PM,242480,19,"AdvaGen Pharma, Ltd",,3,
6,Jul 21 2022 3:03PM,242476,10,ISDIN Corporation,23,,
7,Jul 21 2022 2:53PM,242477,10,Emerginnova,1,,
8,Jul 21 2022 2:51PM,242473,10,ISDIN Corporation,41,,
9,Jul 21 2022 2:49PM,242475,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc.",2,,
1,Jul 22 2022 8:30AM,242486,10,Bio-PRF,4,,
2,Jul 21 2022 3:33PM,242487,12,Hush Hush,,1,
3,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation,2,,
4,Jul 21 2022 3:12PM,242479,10,ISDIN Corporation,24,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc.",2.0,NaN,NaN
1,Jul 22 2022 8:30AM,242486,10,Bio-PRF,4.0,NaN,NaN
2,Jul 21 2022 3:33PM,242487,12,Hush Hush,NaN,1.0,NaN
3,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation,2.0,NaN,NaN
4,Jul 21 2022 3:12PM,242479,10,ISDIN Corporation,24.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc.",2.0,0.0,0.0
1,Jul 22 2022 8:30AM,242486,10,Bio-PRF,4.0,0.0,0.0
2,Jul 21 2022 3:33PM,242487,12,Hush Hush,0.0,1.0,0.0
3,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation,2.0,0.0,0.0
4,Jul 21 2022 3:12PM,242479,10,ISDIN Corporation,24.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3879480,119.0,1.0,1.0
12,484932,0.0,2.0,0.0
16,1212097,9.0,0.0,0.0
19,727378,1.0,9.0,0.0
20,727237,43.0,33.0,20.0
21,969697,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3879480,119.0,1.0,1.0
1,12,484932,0.0,2.0,0.0
2,16,1212097,9.0,0.0,0.0
3,19,727378,1.0,9.0,0.0
4,20,727237,43.0,33.0,20.0
5,21,969697,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,119.0,1.0,1.0
1,12,0.0,2.0,0.0
2,16,9.0,0.0,0.0
3,19,1.0,9.0,0.0
4,20,43.0,33.0,20.0
5,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,119.0
1,12,Released,0.0
2,16,Released,9.0
3,19,Released,1.0
4,20,Released,43.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,20.0,0.0
Executing,1.0,2.0,0.0,9.0,33.0,0.0
Released,119.0,0.0,9.0,1.0,43.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,20.0,0.0
1,Executing,1.0,2.0,0.0,9.0,33.0,0.0
2,Released,119.0,0.0,9.0,1.0,43.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,20.0,0.0
1,Executing,1.0,2.0,0.0,9.0,33.0,0.0
2,Released,119.0,0.0,9.0,1.0,43.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()